In [2]:
from transformers import pipeline

mask_filler = pipeline("fill-mask")

input_sentence = "Hanoi is the <mask> of Vietnam."

predictions = mask_filler(input_sentence, top_k=5)

print(f"Câu gốc: {input_sentence}\n")
for i, pred in enumerate(predictions, 1):
    print(f"{i}. Dự đoán: '{pred['token_str']}' - độ tin cậy: {pred['score']:.4f}")
    print(f"   -> Câu hoàn chỉnh: {pred['sequence']}\n")

No model was supplied, defaulted to distilbert/distilroberta-base and revision fb53ab8 (https://huggingface.co/distilbert/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.
Some weights of the model checkpoint at distilbert/distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


Câu gốc: Hanoi is the <mask> of Vietnam.

1. Dự đoán: ' capital' - độ tin cậy: 0.9341
   -> Câu hoàn chỉnh: Hanoi is the capital of Vietnam.

2. Dự đoán: ' Republic' - độ tin cậy: 0.0300
   -> Câu hoàn chỉnh: Hanoi is the Republic of Vietnam.

3. Dự đoán: ' Capital' - độ tin cậy: 0.0105
   -> Câu hoàn chỉnh: Hanoi is the Capital of Vietnam.

4. Dự đoán: ' birthplace' - độ tin cậy: 0.0054
   -> Câu hoàn chỉnh: Hanoi is the birthplace of Vietnam.

5. Dự đoán: ' heart' - độ tin cậy: 0.0014
   -> Câu hoàn chỉnh: Hanoi is the heart of Vietnam.



In [3]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2")  # hoặc mặc định

prompt = "The best thing about learning NLP is"

generated = generator(
    prompt,
    max_length=60,
    num_return_sequences=3,
    temperature=0.8,
    top_p=0.95
)

print(f"Câu mồi: '{prompt}'\n")
for i, text in enumerate(generated, 1):
    print(f"Kết quả {i}:")
    print(text['generated_text'])
    print("-" * 50)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=256) and `max_length`(=60) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Câu mồi: 'The best thing about learning NLP is'

Kết quả 1:
The best thing about learning NLP is that you can choose your own topics. I have a big library of NLP topics to cover in the NLP course.

I also want to thank you for taking the time to watch my first episode of NLP. If you have any questions, please feel free to leave them in the comments below.

I also want to thank you for reading my NLP book, NLP's Lessons From A Thousand Years of NLP.

You can also follow my NLP Twitter for more NLP related topics and other resources.
--------------------------------------------------
Kết quả 2:
The best thing about learning NLP is that you don't need to understand every aspect of it. You can learn it as you go along.

To start, you should be able to identify the types of learning in question and then determine which parts of it will be most interesting to you. That's one of the reasons why NLP is so useful.

The only way to learn what you know is to learn from it. It's your job to develo

In [4]:
import torch
from transformers import AutoTokenizer, AutoModel

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

sentence = "This is a sample sentence."

inputs = tokenizer(
    sentence,
    padding=True,
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    outputs = model(**inputs)
    last_hidden_state = outputs.last_hidden_state  # (1, seq_len, 768)

# Mean Pooling có attention_mask
attention_mask = inputs['attention_mask'].unsqueeze(-1).expand(last_hidden_state.size()).float()

sum_embeddings = torch.sum(last_hidden_state * attention_mask, dim=1)
sum_mask = torch.clamp(attention_mask.sum(dim=1), min=1e-9)

sentence_embedding = sum_embeddings / sum_mask

print("Vector biểu diễn (một phần đầu):")
print(sentence_embedding[0][:10])  # in 10 giá trị đầu
print(f"\nKích thước vector: {sentence_embedding.shape}")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Vector biểu diễn (một phần đầu):
tensor([-6.3874e-02, -4.2837e-01, -6.6779e-02, -3.8430e-01, -6.5784e-02,
        -2.1826e-01,  4.7636e-01,  4.8659e-01,  4.0647e-05, -7.4273e-02])

Kích thước vector: torch.Size([1, 768])
